# Importing libraries

In [5]:
import torch
import os
import warnings
import evaluate
import platform

import pandas as pd
import numpy as np

from bert_score import score
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling, pipeline
from datasets import Dataset

# Defining file paths

In [6]:
OUTPUT_DIR = "results"
LOG_DIR = "logs"
TRAIN_FILE = "dataset/train.txt"  # Path to your training data file
VAL_FILE = "dataset/val.txt"  # Path to your validation data file
TEST_FILE = "dataset/test.txt"  # Path to your test data file

# Loading Pre-trained Model

In [8]:
tokenizer = AutoTokenizer.from_pretrained("HuggingFaceTB/SmolLM2-360M-Instruct")
model = AutoModelForCausalLM.from_pretrained("HuggingFaceTB/SmolLM2-360M-Instruct")
model.gradient_checkpointing_enable()
model.resize_token_embeddings(len(tokenizer))

tokenizer_config.json:   0%|          | 0.00/3.76k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/801k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.10M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/655 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/724M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

Embedding(49152, 960, padding_idx=2)

# Setting the device

In [8]:
torch.cuda.empty_cache()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

print(device)

cuda


In [12]:
def count_tokens(file_path, tokenizer):
    with open(file_path, "r", encoding="utf-8") as f:
        text = f.read()
    tokens = tokenizer.tokenize(text)
    return len(tokens)

# Count tokens in training and validation files
train_token_count = count_tokens(TRAIN_FILE, tokenizer)
eval_token_count = count_tokens(VAL_FILE, tokenizer)
print(f"Train Tokens: {train_token_count}")
print(f"Validation Tokens: {eval_token_count}")

Train Tokens: 7144
Validation Tokens: 1580


# Training the model

## Setting up training arguments

In [ ]:
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    eval_strategy="epoch",
    logging_strategy="epoch",     # Logs loss at intervals
    learning_rate=1e-5,
    per_device_train_batch_size=1,  # Reduced batch size for limited GPU memory
    per_device_eval_batch_size=1,
    num_train_epochs=10,
    weight_decay=0.01,
    save_total_limit=2,
    logging_dir=LOG_DIR,
    gradient_accumulation_steps=4,  # Accumulate gradients to simulate larger batch sizes
    fp16=True,
    bf16=False
)

## Loading dataset

In [27]:
def load_chatbot_data(file_path):
    """Load and preprocess chatbot data from the given text file."""
    conversations = []
    with open(file_path, "r", encoding="utf-8") as f:
        lines = f.readlines()
        user_input, bot_response = None, None
        for line in lines:
            if line.startswith("User:"):
                user_input = line.replace("User:", "").strip()
            elif line.startswith("Bot:"):
                bot_response = line.replace("Bot:", "").strip()
                if user_input and bot_response:
                    conversations.append({"input": user_input, "output": bot_response})
                    user_input, bot_response = None, None
    return pd.DataFrame(conversations)

In [18]:
df_train = load_chatbot_data(TRAIN_FILE)
df_val = load_chatbot_data(VAL_FILE)
dataset_train = Dataset.from_pandas(df_train)
dataset_val = Dataset.from_pandas(df_val)

print(f"Length of training dataset: {len(df_train)}")
print(f"Length of validation dataset: {len(df_val)}")

Length of training dataset: 96
Length of validation dataset: 24


## Tokenizing dataset

In [28]:
def tokenize_function(examples):
    inputs = [f"{inp} {out}" for inp, out in zip(examples["input"], examples["output"])]
    model_inputs = tokenizer(
        inputs,
        max_length=512,
        truncation=True,
        padding="max_length",
        return_tensors="pt",
    )
    model_inputs["labels"] = model_inputs["input_ids"].clone() 
    return model_inputs

# Apply tokenization
tokenized_train = dataset_train.map(tokenize_function, batched=True)
tokenized_val = dataset_val.map(tokenize_function, batched=True)

Map:   0%|          | 0/96 [00:00<?, ? examples/s]

Map:   0%|          | 0/24 [00:00<?, ? examples/s]

## Data collator 

In [29]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,  # Masked language modeling is not used for causal LM
)

## Compute Metrics

In [30]:
def compute_bertscore(references, candidates, model_type="roberta-large"):
    """Compute BERTScore for chatbot responses."""
    P, R, F1 = score(candidates, references, model_type=model_type, lang="en", rescale_with_baseline=True)
    return {
        "Precision": P.mean().item(),
        "Recall": R.mean().item(),
        "F1 Score": F1.mean().item()
    }

accuracy_metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    """Compute accuracy, BERTScore, and ESAT for evaluation."""
    predictions, labels = eval_pred

    # Decode model outputs
    decoded_preds = [tokenizer.decode(pred, skip_special_tokens=True) for pred in predictions]
    decoded_labels = [tokenizer.decode(label, skip_special_tokens=True) for label in labels]

    # Token-level accuracy
    pred_ids = np.argmax(predictions, axis=-1)
    accuracy = accuracy_metric.compute(predictions=pred_ids, references=labels)

    # Compute BERTScore (similarity of response to ground truth)
    bert_scores = compute_bertscore(decoded_labels, decoded_preds)

    # Combine metrics
    final_metrics = {
        "Accuracy": accuracy["accuracy"],
        **bert_scores,
    }

    print("\nEvaluation Metrics:", final_metrics)
    return final_metrics

## Initialising the model

In [33]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/var/folders/_d/vbh48gmd31s0b7r48gvh7m3w0000gn/T/ipykernel_2229/695427204.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


ValueError: fp16 mixed precision requires a GPU (not 'mps').

## Saving Trained Model

In [12]:
model.save_pretrained("results")
tokenizer.save_pretrained("results")

('results\\tokenizer_config.json',
 'results\\special_tokens_map.json',
 'results\\vocab.json',
 'results\\merges.txt',
 'results\\added_tokens.json',
 'results\\tokenizer.json')

# Evaluating the model

## Load trained model

In [9]:
model = AutoModelForCausalLM.from_pretrained("results", device_map="auto")
tokenizer = AutoTokenizer.from_pretrained("results", device_map="auto")

model.to(device)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(49152, 960, padding_idx=2)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=960, out_features=960, bias=False)
          (k_proj): Linear(in_features=960, out_features=320, bias=False)
          (v_proj): Linear(in_features=960, out_features=320, bias=False)
          (o_proj): Linear(in_features=960, out_features=960, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=960, out_features=2560, bias=False)
          (up_proj): Linear(in_features=960, out_features=2560, bias=False)
          (down_proj): Linear(in_features=2560, out_features=960, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((960,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((960,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((960,), eps=1e-05)
    (rotary_emb)

In [ ]:
# def chatbot_response(user_input, chatbot):
#     sentiment = get_sentiment(user_input)  # Get sentiment from BERT

#     # Define chatbot responses based on sentiment
#     responses = {
#         "positive": "I'm happy to hear that! Keep up the great energy. 😊",
#         "neutral": "That sounds interesting. Tell me more, I'm here to listen. 🧡",
#         "negative": "I'm sorry you're feeling this way. You're not alone—would you like some self-care tips? 💙"
#     }

#     # Modify chatbot input based on sentiment
#     modified_input = f"{user_input} [Sentiment: {sentiment.capitalize()}]"

#     # Get chatbot response
#     bot_reply = chatbot.generate_response(modified_input)  # Assuming chatbot has a generate_response method

#     # Modify response based on sentiment if needed
#     if sentiment == "negative":
#         bot_reply += " Remember, you're not alone. Would you like some coping tips? 💙"

#     return bot_reply

In [10]:
pipe = pipeline("translation", model="Helsinki-NLP/opus-mt-en-zh")
sentiment_classifier = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")

c:\Users\wongq\anaconda3\envs\python3.9\lib\site-packages\transformers\models\marian\tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
Device set to use cuda:0
Device set to use cuda:0


In [11]:
def get_sentiment(text):
    """Improved sentiment analysis with keyword-based distress detection."""
    crisis_keywords = ["end my life", "suicide", "don't want to live", "kill myself", "worthless", "no reason to live"]

    # Check if crisis words are in the input
    if any(phrase in text.lower() for phrase in crisis_keywords):
        return "crisis"  # Override sentiment if crisis words are detected

    # Otherwise, use DistilBERT-based sentiment analysis
    result = sentiment_classifier(text)[0]
    label = result['label']

    # Convert to sentiment categories based on DistilBERT outputs
    if label == "NEGATIVE":
        return "negative"
    elif label == "POSITIVE":
        return "positive"
    else:
        return "neutral"

In [ ]:
# def chatbot_response(prompt, max_length=200):
#     system_prompt = "You are a helpful and supportive chatbot. Answer the user's question in a clear and concise way without repeating their words exactly."
#     full_prompt = f"{system_prompt}\nUser: {prompt}\nBot:"

#     sentiment_results = get_sentiment(prompt)

#     inputs = tokenizer(full_prompt, return_tensors="pt")
#     inputs = {key: val.to(device) for key, val in inputs.items()}

#     outputs = model.generate(**inputs, max_length=max_length)
#     response = tokenizer.decode(outputs[0], skip_special_tokens=True)

#     # Ensure the response does not include the system prompt
#     response = response.replace(system_prompt, "").strip()
    
#     # Remove any leftover system prompt instructions
#     if "Bot:" in response:
#         response = response.split("Bot:")[-1].strip()

#     # Translate response to Chinese
#     translated = pipe(response)[0]['translation_text']

#     return response, translated, sentiment_results

In [12]:
def chatbot_response(prompt, max_length=200):
    system_prompt = "You are a helpful and supportive chatbot. Answer the user's question in a clear and concise way without repeating their words exactly."
    full_prompt = f"{system_prompt}\nUser: {prompt}\nBot:"

    sentiment_results = get_sentiment(prompt)

    inputs = tokenizer(full_prompt, return_tensors="pt")
    inputs = {key: val.to(device) for key, val in inputs.items()}

    outputs = model.generate(
        **inputs, 
        max_length=max_length,
        repetition_penalty=1.3,  # Penalize repeated words/phrases
        no_repeat_ngram_size=3,  # Prevent 3-word repetition
        temperature=0.7,  # Lower randomness to keep responses meaningful
        top_p=0.9,  # Diverse sampling to avoid looping responses
        top_k=50  # Limits token choices to avoid repetition
    )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Ensure the response does not include the system prompt
    response = response.replace(system_prompt, "").strip()
    
    # Remove any leftover system prompt instructions
    if "Bot:" in response:
        response = response.split("Bot:")[-1].strip()

    # Translate response to Chinese
    translated = pipe(response)[0]['translation_text']

    return response, translated, sentiment_results

In [13]:
if __name__ == "__main__":
    print("Chatbot is ready! Type 'exit' to stop.")
    while True:
        user_input = input("User: ")
        if user_input.lower() == "exit":
            break
        response, translated, sentiment_results = chatbot_response(user_input)
        print(f"User: {user_input}")
        print(f"Bot: {response}")
        print(f"Translated Text: {translated}")
        print(f"Sentiment Results: {sentiment_results}")

Chatbot is ready! Type 'exit' to stop.


c:\Users\wongq\anaconda3\envs\python3.9\lib\site-packages\transformers\generation\configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
c:\Users\wongq\anaconda3\envs\python3.9\lib\site-packages\transformers\generation\configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


User: i think my mental health is getting better 
Bot: I'm glad to hear that you're feeling more positive about your well-being! It takes time, but small steps can make big differences over time. What specific areas of mind wellness would like me to help with?
Translated Text: 我很高兴听到你对自己的福祉感到更积极!这需要时间,但小步骤会随着时间而产生很大的差异。什么具体领域的心智健康需要我帮助?
Sentiment Results: negative
User: i do not think about harming myself anymore these days
Bot: I'm sorry to hear that you're feeling this way, but it can be really hard when thoughts of self-harm come up unexpectedly. It might help if we talked more openly with each other or sought support from someone who understands what you need. You deserve care and happiness. If there is something specific going on for which you feel like talking, please reach out. We all go through tough times sometimes.
Translated Text: 我很遗憾听到你这样想, 但突然出现自我伤害的想法时会很难。 如果我们彼此更公开地交谈或寻求理解你需要的东西的人的支持, 可能会有帮助。 你应该得到关心和幸福。 如果有具体的事情发生, 你愿意与之交谈, 请伸出援手。 我们有时会经历艰难的时刻 。
Sentiment Results: posi